In [244]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer

In [245]:
def robustScalerHelper(arr):
    p1 = np.nanpercentile(arr, 0.1)
    p99 = np.nanpercentile(arr, 99.9)
    out = (arr - p1) / (p99 - p1)
    out -= np.nanmin(out)
    out[np.where(out>1)] = 1
    return out

def robustScaler(arr):
    arr -= np.nanmin(arr)
    if np.nansum(arr[np.where(arr > np.nanpercentile(arr,99))]) > np.nansum(arr[np.where(np.nanpercentile(arr,99) > arr)]):
        #print('log')
        return robustScalerHelper(np.log(arr))
    else:
        #print('lin')
        return robustScalerHelper(arr)

In [246]:
dataset = pd.read_csv('datasets/TOI_2025.11.13_04.26.29.csv', comment='#', skiprows=lambda x: x < 4)
dataset['tfopwg_disp'] = dataset['tfopwg_disp'].replace({'KP': 'P', 'CP': 'P', 'FA': 'F', 'FP': 'F', 'APC':'PC'})
dataset = dataset.rename(columns={'tfopwg_disp': 'isExoplanet'})

In [247]:
for i in np.unique(dataset['isExoplanet']):
    print(f'{np.array(np.array(dataset['isExoplanet'])[np.where(dataset['isExoplanet'] == i)]).size} values have {i}')

1299 values have F
1283 values have P
5128 values have PC


In [248]:
print(dataset.shape)
print(dataset.dropna().shape)

(7710, 66)
(0, 66)


In [249]:
for i in ['rastr','decstr','toi_created','rowupdate','st_raderr1','st_raderr2','st_loggerr1','st_loggerr2','pl_radeerr1','pl_radeerr2']:
    dataset.drop(i,axis=1,inplace=True)
for i in dataset.keys():
    try:
        dataset[i] = np.array(robustScaler(np.array(dataset[i])))
    except:
        print(i)
nanCount = dataset.isna().sum()
print(nanCount)
for i in nanCount.keys():
    if nanCount[i] > 5000:
        dataset.drop(i,axis=1,inplace=True)

isExoplanet
loc_rowid             0
toi                   0
tid                   0
isExoplanet           0
ra                    0
dec                   0
st_pmra             134
st_pmraerr1         134
st_pmraerr2         134
st_pmralim         7710
st_pmdec            134
st_pmdecerr1        134
st_pmdecerr2        134
st_pmdeclim        7710
pl_tranmid            0
pl_tranmiderr1       11
pl_tranmiderr2       11
pl_tranmidlim      7710
pl_orbper           108
pl_orbpererr1       131
pl_orbpererr2       131
pl_orbperlim       7710
pl_trandurh           0
pl_trandurherr1      13
pl_trandurherr2      13
pl_trandurhlim     7710
pl_trandep            0
pl_trandeperr1        6
pl_trandeperr2        6
pl_trandeplim      7710
pl_rade             506
pl_radelim         7710
pl_insol            176
pl_insolerr1       7710
pl_insolerr2       7710
pl_insollim        7710
pl_eqt              311
pl_eqterr1         7710
pl_eqterr2         7710
pl_eqtlim          7710
st_tmag               0
st_t

/tmp/ipykernel_134464/3042963735.py:4: RuntimeWarning: invalid value encountered in divide
  out = (arr - p1) / (p99 - p1)
/tmp/ipykernel_134464/3042963735.py:5: RuntimeWarning: All-NaN slice encountered
  out -= np.nanmin(out)
/tmp/ipykernel_134464/3042963735.py:13: RuntimeWarning: divide by zero encountered in log
  return robustScalerHelper(np.log(arr))
/tmp/ipykernel_134464/3042963735.py:5: RuntimeWarning: invalid value encountered in subtract
  out -= np.nanmin(out)
/tmp/ipykernel_134464/3042963735.py:10: RuntimeWarning: All-NaN slice encountered
  arr -= np.nanmin(arr)
/home/thomas/anaconda3/envs/brain/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1409: RuntimeWarning: All-NaN slice encountered
  return _nanquantile_unchecked(


In [250]:
dataset = dataset.sort_values('isExoplanet')
labels = dataset['isExoplanet']
data = dataset.drop('isExoplanet',axis=1)
lab = data.keys()

In [251]:
imputer = KNNImputer(n_neighbors=5)
imputed = imputer.fit_transform(data)

In [252]:
imputed.shape

(7710, 37)

In [253]:
labels.shape

(7710,)

In [254]:
im = pd.DataFrame({'isExoplanet':labels} | {i:j for i,j in zip(lab,imputed.T)})

In [255]:
im

,isExoplanet,loc_rowid,toi,tid,ra,dec,st_pmra,st_pmraerr1,st_pmraerr2,st_pmdec,...,st_tmagerr1,st_tmagerr2,st_dist,st_disterr1,st_disterr2,st_teff,st_tefferr1,st_tefferr2,st_logg,st_rad
0,F,0.000000,0.121493,0.025595,0.312370,0.441976,1.0,0.014996,1.0,1.0,...,0.021821,1.0,0.064991,1.0,1.0,0.701740,0.041584,1.0,1.0,0.235035
2828,F,0.367579,0.450701,0.067247,0.331867,0.290722,1.0,0.008355,1.0,1.0,...,0.011750,1.0,0.075878,1.0,1.0,0.202685,0.015959,1.0,1.0,0.363145
2827,F,0.367449,0.450566,0.121039,0.295884,0.229987,1.0,0.003428,1.0,1.0,...,0.010071,1.0,0.037440,1.0,1.0,0.152683,0.018787,1.0,1.0,0.072645
2815,F,0.365889,0.449079,0.144183,0.358118,0.357285,1.0,0.004713,1.0,1.0,...,0.010071,1.0,0.133968,1.0,1.0,0.359782,0.018963,1.0,1.0,0.188159
2814,F,0.365759,0.448944,0.073676,0.339228,0.335514,1.0,0.006427,1.0,1.0,...,0.026857,1.0,0.545757,1.0,1.0,0.640394,0.013020,1.0,1.0,0.246945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3512,PC,0.456484,0.533679,0.130036,0.853502,0.907855,1.0,0.004713,1.0,1.0,...,0.011750,1.0,0.121624,1.0,1.0,0.297822,0.018787,1.0,1.0,0.167572
485,PC,0.063040,0.176767,0.058837,0.945213,0.710155,1.0,0.211011,1.0,1.0,...,0.010071,1.0,0.010402,1.0,1.0,0.311591,0.039970,1.0,1.0,0.300242
3510,PC,0.456224,0.533544,0.131769,0.817088,0.896475,1.0,0.005784,1.0,1.0,...,0.016786,1.0,0.047101,1.0,1.0,0.202289,0.018787,1.0,1.0,0.077416
3537,PC,0.459734,0.536652,0.043431,0.716845,0.724912,1.0,0.460154,1.0,1.0,...,0.010071,1.0,0.115809,1.0,1.0,0.288731,0.067623,1.0,1.0,0.109701


In [256]:
labeled = im[~im['isExoplanet'].isin(['PC', 'APC'])]
unlabeled = im[im['isExoplanet'].isin(['PC', 'APC'])]

In [257]:
for i in np.unique(im['isExoplanet']):
    print(f'{np.array(np.array(im['isExoplanet'])[np.where(im['isExoplanet'] == i)]).size} values have {i}')

1299 values have F
1283 values have P
5128 values have PC
